In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# Import pdlmfit

try:
    # If pdlmfit is in PYTHONPATH (or is installed), then use a direct import
    import pdlmfit
    
except ImportError:
    # Attempt to demo pdlmfit without installing by importing from directory
    pdlmfit_path = '../../pdlmfit'
    
    print("Module pdlmfit was not found in PYTHONPATH. Looking for module in directory '{:s}'".format(pdlmfit_path))
    
    if os.path.exists(pdlmfit_path):
        import imp
        pdlmfit = imp.load_package('pdlmfit', pdlmfit_path)
        print("Module pdlmfitr was found in the directory '{:s}' and imported.".format(pdlmfit_path))
    else:
        raise ImportError("Module pdlmfit could not be found in the directory '{:s}'.".format(pdlmfit_path) + \
        "This demonstration will not run until the module is located.")

Module pdlmfit was not found in PYTHONPATH. Looking for module in directory '../../pdlmfit'
Module pdlmfitr was found in the directory '../../pdlmfit' and imported.


In [3]:
data = pd.read_csv('GCN4_fourfield_R2.tsv', sep='\t')
#data = data.loc[(data.resi<=10)&(data.field<17.0)].reset_index(drop=True)

In [4]:
def exponential_decay(par, xdata):
    
    # Parse multiple input parameter
    # formats for intensity, rate    
    if hasattr(par,'valuesdict'):
        # lmfit parameter format
        var = par.valuesdict()
        inten = var['inten']
        rate = var['rate']
    elif hasattr(par,'keys'):
        # dict format
        inten = par['inten']
        rate = par['rate']
    else:
        # array/list/tuple format
        inten = par[0]
        rate = par[1]

    # Calculate the y-data from the parameters
    return inten * np.exp(-1*rate*xdata)

In [5]:
func = exponential_decay
groupcols = ['resi', 'field']
xname = 'time'
yname = 'intensity'
yerr = None

method = 'leastsq'
sigma = [0.95]
threads = 4

params = [{'name':'inten', 'value':np.asarray(data.groupby(groupcols)[yname].max()), 'vary':True},
          {'name':'rate', 'value':20.0, 'vary':True}]

In [6]:
%autoreload 1
a = pdlmfit.pdlmfit(data, exponential_decay, groupcols, params, 
                  xname, yname, 
                  method=method, sigma=sigma, threads=threads)
a.fit()

ERROR: Line magic function `%autoreload` not found.
/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/pandas/core/indexing.py:1294: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [7]:
a.results

inten                                  rate            \
                   value       stderr        ci0.95      value    stderr   
resi field                                                                 
5    14.1   1.531337e+04   139.475597    232.504669   5.085937  0.153085   
     16.5   6.869230e+04   362.676887    618.930303   5.510422  0.132718   
     18.8   3.188525e+05   940.572232   1593.726552   6.988834  0.096050   
     21.1   8.319324e+05  1428.215703   2543.892832   5.950005  0.032391   
6    14.1   1.215004e+04   125.957061    210.737126   5.913539  0.227691   
     16.5   6.733715e+04   336.018198    572.242574   5.697662  0.130639   
     18.8   2.578649e+05   555.307822    905.503586   6.247189  0.041395   
     21.1   7.680774e+05  1923.626547   2928.719491   6.318552  0.036332   
7    14.1   1.469795e+04   167.021450    279.988834   6.200861  0.268259   
     16.5   7.726157e+04   333.556144    552.803867   5.310241  0.080372   
     18.8   3.095596e+05   780.355307   1338.118371   7.496154  0.094357   
     21.1   1.061808e+06  3633.357495   5571.244409   6.727924  0.051949   
8    14.1   1.384054e+04   132.637233    221.585751   6.242079  0.221234   
     16.5   7.410068e+04   486.329832    836.744125   7.753889  0.145327   
     18.8   3.049545e+05  1080.804121   1818.653469   7.589037  0.100657   
     21.1   9.079491e+05  1526.207279   2707.262026   7.704772  0.041132   
9    14.1   1.217114e+04   104.751966    174.967962   6.830633  0.215821   
     16.5   6.405526e+04   322.204238    561.149535   7.071610  0.094696   
     18.8   2.508264e+05   925.367570   1560.371991   8.286205  0.115279   
     21.1   7.867222e+05  2380.790984   3478.012406   8.335381  0.043953   
10   14.1   1.013809e+04    81.913608    136.751085   7.178645  0.204430   
     16.5   5.086888e+04   212.575306    366.034028   6.945686  0.127468   
     18.8   1.903791e+05   691.417195   1177.565529   8.571641  0.172095   
     21.1   5.609603e+05   784.645950   1305.888739   8.104072  0.041178   
11   14.1   1.261758e+04   112.077186    187.306071   7.172064  0.242344   
     16.5   6.193936e+04   248.334209    432.205291   7.012452  0.088731   
     18.8   2.736030e+05   584.586122    946.484610   8.303567  0.066509   
     21.1   7.456061e+05  1445.801559   2509.490488   8.232976  0.035887   
12   14.1   8.860524e+03    78.404563    130.048644   7.886431  0.240091   
     16.5   4.235079e+04   205.418070    346.924347   7.197420  0.188525   
...                  ...          ...           ...        ...       ...   
51   18.8   2.391296e+04   364.380347    564.935908  21.579354  0.808148   
     21.1   6.116037e+04  1219.832672   1835.846020  21.226526  0.953968   
52   14.1   1.774190e+03    13.183360     20.842506  23.972711  0.358795   
     16.5   1.434005e+04    92.731668    128.740736  23.757466  0.752718   
     18.8   2.311306e+04   240.777337    414.488539  28.447961  0.943940   
     21.1   7.239204e+04   305.313948    488.724447  32.172882  0.537314   
53   14.1   2.533089e+03    37.012709     50.079446  19.947132  1.482077   
     16.5   1.989379e+04   126.577805    216.226653  20.059566  0.535657   
     18.8   4.338765e+04   258.917461    421.099120  24.852374  0.613806   
     21.1   1.194846e+05   201.128366    340.749959  27.317235  0.126948   
54   14.1   2.386309e+03    13.720976     22.826418  23.112244  0.388829   
     16.5   1.775208e+04   237.841514    244.760542  21.957915  1.547561   
     18.8   4.047259e+04   149.483291    244.835145  27.263036  0.387252   
     21.1   9.777623e+04   629.865793   1117.527149  28.199335  0.691295   
55   14.1   2.136555e+03    14.356287     23.283920  21.531813  0.379501   
     16.5   1.633872e+04    83.557561    141.295310  20.382636  0.213037   
     18.8   3.548488e+04   277.119168    457.214591  27.162083  0.400621   
     21.1   9.964350e+04   688.483265   1049.194812  29.111896  0.372150   
56   14.1   4.988408e+03    22.787719     38.686815  14.001320  0.2

In [34]:
def predict(self, df, xcalc=False, xtype='global', xnum=50):
    
    # Get the index for selection
    index = df.index.unique()[0]
    
    # Choose the xdata
    xname = self._xname
    xdata = self.data.loc[index, xname]
    
    if xcalc:
        if xtype == 'global':
            xmin = self.data[xname].min()
            xmax = self.data[xname].max()
        else:
            xmin = xdata.min()
            xmax = xdata.max()
            
        xname = 'xdata'
        xdata = pd.Series(np.linspace(xmin, xmax, xnum))
        xindex = pd.Index(np.array([index]*xnum))
    
    # Pick out the parameters for this data
    params = ( self
             .results
             .sortlevel(axis=1)
             .loc[[index], (slice(None),['value'])]
             )
    params.columns = params.columns.levels[0]
    params = params[self._paramnames].squeeze()
    
    
    model_eq = self._fitobj.model_eq.loc[index]
    
    ycalc = model_eq(params, xdata)
    
    if not xcalc:
        return ycalc
    else:
        return pd.DataFrame({xname:xdata, 'ycalc':ycalc})




z = ( a
     .data
     .groupby(level=range(a._ngroupcols), group_keys=False, as_index=False)
     .apply(lambda x: predict(a, x, True, xnum=5))
     )
z

,xdata,ycalc
0,0.004,15004.983475
1,0.055,11576.759543
2,0.106,8931.790011
3,0.157,6891.122901
4,0.208,5316.691814
0,0.004,67194.774141
1,0.055,50732.360278
2,0.106,38303.162892
3,0.157,28919.062301
4,0.208,21834.023647
